# 로그파일 분석하기

- 파일: .log → .txt  
- 환경: Google Colab

처음에 아무 이상없이 log파일을 읽어오고 splitline()도 정상적으로 작동해서 dataframe구성  
pandas profiling을 해보려고 하니 concat() got an unexpected keyword argument 'join_axes' 발생  
해결책으로 업그레이드(https://stackoverflow.com/questions/60839909/errors-running-pandas-profile-report)하고 다른 패키지를 건드린 후  
런타임을 다시 시작하는 과정에서 dataframe 사라짐  
그 후 다시 파일을 읽어오려고 하니 codec utf-8 관련 오류 발생  
우선 읽어올 때 rb로 처리하도록 하면 읽어오기는 하지만 여전히 텍스트를 분리할 때 오류 발생  
어떤 행에서 오류가 발생하는지 봤더니 한국어 포함 행에서 오류 발생  
자동으로 인코딩 감지해서 읽어오는 옵션을 빌려왔는데 여전히 오류 발생  
.log 파일을 인코딩 utf-8로 지정한 텍스트 파일로 바꿔 저장해서 업로드하니 해결

In [ ]:
# 판다스 프로파일링을 Colab에서 쓰려고 업데이트
!pip3 install pandas_profiling --upgrade

     |████████████████████████████████| 262 kB 5.2 MB/s 
     |████████████████████████████████| 10.9 MB 48.5 MB/s 
     |████████████████████████████████| 4.7 MB 25.9 MB/s 
     |████████████████████████████████| 690 kB 53.5 MB/s 
     |████████████████████████████████| 596 kB 55.9 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 102 kB 11.9 MB/s 
     |████████████████████████████████| 812 kB 50.8 MB/s 
     |████████████████████████████████| 38.1 MB 1.1 MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=5a1c0b2d5766cc3939d187379ab89e9fdca08e5f1f0f1896bf84b1bb4caa5d29
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295206 sha256=9c69ba0d5c5d76d7add774dd24d0fc7167d493995d6b79e407ddc9c71cc1742e
  Stored in directory: /root/.cache/pip/wheels/4c/d5/59/5e3

In [ ]:
# MarkupSafe를 다운그레이드해야 정상적으로 표기된다고 해서 다운그레이드
pip install MarkupSafe==2.0.1

  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
# 패키지 불러오기
import pandas as pd
from tqdm.notebook import tqdm
import pandas_profiling

In [ ]:
# 이미 로그 파일을 읽고 데이트프레임을 만든 경우
log_date_df = pd.read_csv('filesdataframe.csv')

In [ ]:
# 인코딩을 utf-8로 지정하여 로그 파일 읽고 행으로 분리하고 연결 닫기
with open('files.txt', 'r', encoding='utf-8') as f:
  lines = f.read().splitlines()
  f.close()

In [ ]:
# 파일 내용 확인
lines[:10] # 보안상 출력 삭제

In [ ]:
# 빈칸 구분자로 list comprehesion → dataframe 구성
log_date_df = pd.DataFrame([line.split(' ') for line in lines[4:]], columns = lines[3].split(' ')[1:])

In [ ]:
# csv파일로 저장
log_date_df.to_csv('filesdataframe.csv')

In [ ]:
# 판다스 프로파일링 실행
pr = log_date_df.profile_report()
pr # 보안상 출력 삭제

# Appendix

In [ ]:
# 시간이 매우 오래 걸림
for line in tqdm(lines[4:]):
    # print(line.split('\t'))
    # print(len(line.split('\t')))
    # users_log_raw = pd.concat([users_log_raw, pd.Series(line.split(' '))], axis=1)
    # users_log_raw = users_log_raw.append(pd.DataFrame([line.split(' ')]))
    [line.split(' ')]


  0%|          | 0/101352 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
pr = pandas_profiling.ProfileReport(log_date_df)
pr

TypeError: ignored

In [ ]:
users_log_raw # 보안상 출력 삭제

In [ ]:
# pandas_profiling.__version__
!pip freeze |grep pandas-profiling

pandas-profiling==1.4.1


In [ ]:
file = open('files.log', 'r')

In [ ]:
# with open('files.log', 'r', encoding='cp949') as f: 'cp949' codec can't decode byte 0xec in position 6303967: illegal multibyte sequence
# with open('files.log', 'r', encoding='utf-16') as f: UTF-16 stream does not start with BOM
# with open('files.log', 'r', encoding=enc) as f: 'charmap' codec can't decode byte 0x90 in position 5926451: character maps to <undefined>
with open('files.log', 'r', encoding=enc) as f: 
    # lines = f.readline()
    lines = f.read().splitlines()
    f.close()

UnicodeDecodeError: ignored

In [ ]:
lines = file.read().splitlines()
file.close()
#pandas profiling 업데이트하고 런타임 재시작하니 갑자기 'utf-8' codec can't decode byte 0xbc in position 52441641: invalid start byte

UnicodeDecodeError: ignored

In [ ]:
lines[3].decode()

'#Fields: date time cs-method cs-uri-stem cs-uri-query cs-username c-ip cs(User-Agent) cs(Referer) sc-status sc-substatus sc-win32-status sc-bytes cs-bytes time-taken'

In [ ]:
# users_log_raw = pd.DataFrame(columns = lines[3].split(' ')[1:]) a bytes-like object is required, not 'str'
users_log_raw = pd.DataFrame(columns = lines[3].decode().split(' ')[1:])

In [ ]:
log_date_df = pd.DataFrame([line.decode().split(' ') for line in lines[4:]], columns = lines[3].decode().split(' ')[1:])

UnicodeDecodeError: ignored

In [ ]:
lines[148636].decode().split(' ')

UnicodeDecodeError: ignored

In [ ]:
for line in tqdm(lines[4:]):
    print(line.decode().split(' ')) # 어느 행에서 오류 발생하는지 확인

In [ ]:
lines[148636] # 내용: 한국어 포함 jpg 파일

IndexError: ignored

In [ ]:
import chardet

In [ ]:
# 파일을 읽은 후 어떤 인코딩으로 되어 있는지 감지: 시간이 매우 오래 걸림(5분)
rawdata = open('files.log', 'rb').read()
result = chardet.detect(rawdata)
enc = result['encoding']

In [ ]:
enc

'Windows-1254'